# Applied Data Science Capstone - Week 3

## Assignment: Segmenting and Clustering Neighborhoods in Toronto
### Note: This notebook contains all the three tasks.

### Task 1:
#### Transform the data in the table on the Wikipedia page into the above pandas dataframe.
####  Wikipedia page - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Download all the dependencies and setup imports

In [2]:
# install 'lxml' python package to be able to read table HTML tables from Pandas
!conda install -c conda-forge lxml --yes
# Geocoder Python package to get latitude & longitude from postal code
!conda install -c conda-forge geocoder --yes
# Library to handle geo-locations
!conda install -c conda-forge geopy --yes
# map rendering library
!conda install -c conda-forge folium=0.5.0 --yes

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import geocoder # convert postal code into latitude and longitude values

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages 

#### Load the table into dataframe

In [3]:
wiki_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df_wiki = pd.read_html(wiki_page, header=0)[0]
print(df_wiki.shape)
df_wiki.head()

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
df_wiki = df_wiki[df_wiki['Borough'] != 'Not assigned'].reset_index(drop=True)
print(df_wiki.shape)
df_wiki.head()

(211, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### If more than one neighborhood exists for one postal code, combine them into one with ',' as delimmiter.

In [5]:
df_wiki = df_wiki.groupby(['Postcode','Borough'], sort=False).agg( ', '.join).reset_index()
df_wiki.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


#### If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough.

In [6]:
df_wiki.loc[df_wiki['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df_wiki['Borough']
df_wiki.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [7]:
df_wiki.shape

(103, 3)

### Task 2:
#### Add the latitude and the longitude coordinates of each neighborhood using 'Geocoder' package.

In [18]:
# function to get latitude and longitude from postal code using geocoder package
def get_lat_lng(postal_code):
    # initialize your variable to None
    lat_lng = None
    # loop until you get the coordinates
    while(lat_lng is None):
        # g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))  # if ran out of limit
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng = g.latlng
    return lat_lng
    
get_lat_lng('M5G')

[43.65609081300005, -79.38492999999994]

Retrive coordinates for all the postal codes in the dataframe.

In [28]:
df_wiki[['Latitude', 'Longitude']] = df_wiki['Postcode'].apply(lambda postal_code: pd.Series(get_lat_lng(postal_code), index=['Latitude', 'Longitude']))
df_wiki.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035


In [29]:
df_wiki.shape

(103, 5)

### Task 3:
#### Explore, Analyse and Cluster the neighborhoods in Toronto.